## File for investigations. Model is in MusicGenerator

In [1]:
from scipy.io.wavfile import read,write
import matplotlib.pyplot as plt
import librosa
import librosa.display
import IPython.display as ipd
import numpy as np
import pandas as pd
import os
from keras.models import Sequential,Model
from keras.layers import Dense,LSTM,Flatten,Lambda,Reshape,Input,InputLayer
import tensorflow as tf

Using TensorFlow backend.


In [2]:
rate,song1=read("F:\BSU_FAMCS\Learning_Courses\musicGenerator\dataset\WAV\Billie Eilish - bury a friend_(mp3type.ru).wav")

In [3]:
rate

44100

In [4]:
type(song1)

numpy.ndarray

In [5]:
song1.shape

(8517620, 2)

In [6]:
sample_size=1000000
song1pd=pd.DataFrame(song1[:sample_size,:])

In [7]:
song1pd.tail()

,0,1
999995,-8963,-6965
999996,-8900,-6834
999997,-8777,-6623
999998,-8698,-6439
999999,-8996,-6657


In [3]:
#Adding only 1million for every song
path="F:\BSU_FAMCS\Learning_Courses\musicGenerator\dataset\WAV"
time_steps=50
song_max_length_train=1500+1

In [4]:
Xtrain1,Xtrain2,Ytrain1,Ytrain2=[],[],[],[]

for song_name in os.listdir(path):
    
    rate,song=read(path+'\\'+song_name)
    
    for i in range(0,song_max_length_train,time_steps):
        #For the first channel in audio
        Xtrain1.append(song[i:i+time_steps,0])
        Ytrain1.append(song[i+1:i+1+time_steps,0])
        #For the second channel in audio
        Xtrain2.append(song[i:i+time_steps,1])
        Ytrain2.append(song[i+1:i+1+time_steps,1])
        
Xtrain1=np.array(Xtrain1)
Xtrain2=np.array(Xtrain2)        
Ytrain1=np.array(Ytrain1)        
Ytrain2=np.array(Ytrain2)        

In [6]:
#Reshaping 2D->3D
        
Xtrain1=Xtrain1.reshape((-1,1,time_steps))
Xtrain2=Xtrain2.reshape((-1,time_steps,1))     
Ytrain1=Ytrain1.reshape((time_steps,-1,1))
Ytrain2=Ytrain2.reshape((time_steps,-1,1))

In [7]:
m=Xtrain1.shape[0]
print(m)

930


In [8]:
Ytrain1.shape

(50, 930, 1)

In [9]:
Xtrain1Ravel=np.ravel(Xtrain1)

In [10]:
unique_notes=len(set(list(Xtrain1Ravel)))
unique_notes

3000

In [11]:
#Global variables
dimension_LSTM=256
LSTM_unit=LSTM(dimension_LSTM,return_state=True)
reshapor=Reshape((1,1))
densor1=Dense(256,activation="tanh")
densor2=Dense(128,activation="tanh")
densor3=Dense(64,activation="linear")
densor4=Dense(1,activation="linear")

In [12]:
def get_model():
    
    X=Input(shape=(time_steps,1))
    
    a0=Input(shape=(dimension_LSTM,),name="a0")
    c0=Input(shape=(dimension_LSTM,),name="c0")
    
    a=a0
    c=c0
    outputs=[]
    
    for t in range(time_steps):
        x=Lambda(lambda z: z[:t:])(X)
        #x=reshapor(x)
        
        a,_,c=LSTM_unit(inputs=x,initial_state=[a,c])
        
        y=densor1(a)
        #y=densor2(y)
        y=densor3(y)
        out=densor4(y)

        outputs.append(out)
    
    model=Model(inputs=[X,a0,c0],outputs=outputs)
    return model

In [13]:
model_first=get_model()


In [14]:
model_first.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 50, 1)        0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 50, 1)        0           input_1[0][0]                    
__________________________________________________________________________________________________
a0 (InputLayer)                 (None, 256)          0                                            
__________________________________________________________________________________________________
c0 (InputLayer)                 (None, 256)          0                                            
____________________________________________________________________________________________

In [17]:
model_first.compile(optimizer="adam",loss="mean_absolute_error",metrics=["accuracy"])

In [18]:
batch_size=64
a0=np.zeros((m,dimension_LSTM))
c0=np.zeros((m,dimension_LSTM))

In [19]:
history=model_first.fit([Xtrain1,a0,c0],list(Ytrain1),epochs=5,batch_size=batch_size)

ValueError: Error when checking input: expected input_1 to have shape (50, 1) but got array with shape (1, 50)

In [21]:
print(history)

NameError: name 'history' is not defined

#Model 1 for the first channel
model1=Sequential()
model1.add(InputLayer(input_shape=(time_steps,1)))
model1.add(LSTM(units=256,activation="tanh"))
model1.add(Dense(units=256,activation="tanh"))
model1.add(Dense(units=128,activation="tanh"))
model1.add(Reshape((128,-1)))
model1.add(Dense(units=time_steps,activation="sigmoid"))

model1.summary()

model1.compile(optimizer="adam",loss="mean_squared_error",metrics=["accuracy"])

model1.fit(Xtrain1,Ytrain1,epochs=10,batch_size=1)

## Librosa audio displaying

In [ ]:
song2,sr=librosa.load("F:\BSU_FAMCS\Learning_Courses\musicGenerator\dataset\WAV\Billie Eilish - bury a friend_(mp3type.ru).wav")

In [ ]:
print(song2.shape)

In [ ]:
sr

In [ ]:
plt.figure(figsize=(13,5))
librosa.display.waveplot(song2[:500000],sr=sr)

In [ ]:
#Spectrogramme
X=librosa.stft(song2)
Xdb=librosa.amplitude_to_db(abs(X))
plt.figure(figsize=(13,5))
librosa.display.specshow(Xdb,sr=sr,x_axis="time",y_axis="hz")

In [ ]:
#Playing music
ipd.Audio("F:\BSU_FAMCS\Learning_Courses\musicGenerator\dataset\WAV\Billie Eilish - bury a friend_(mp3type.ru).wav")